## 네이버 리뷰 크롤링
* **입력값 예시**
    * 성수동 놀거리
    * 성수동 맛집 

* **가져와야할 데이터**
    * 가게 이름 
    * 카테고리 
    * 주소
    * 리뷰 개수 
    * 리뷰 평점 
    * 방문자 리뷰 

In [35]:
import os
import sys
import urllib.request
import pandas as pd
import json
import re
import requests
import time

In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [98]:
options = Options()
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
options.add_argument("--disable-dev-shm-usage")  # 리소스 제한 방지

driver = webdriver.Chrome(options=options, keep_alive=True)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [99]:
# iframe이 2개
# 가게 리스트 iframe = searchIframe
# 각 가게에 대한 정보가 가게리스트 옆에 iframe으로 뜬다. 
# 각 가게에 대한 정보 iframe = entryIframe
def store_list_iframe():
    driver.switch_to.default_content()
    time.sleep(2)
    iframe = driver.find_element(By.ID, "searchIframe")
    driver.switch_to.frame(iframe)

def store_info_iframe():
    driver.switch_to.default_content()
    time.sleep(2) # 로딩 시간 필요함 
    iframe = driver.find_element(By.XPATH, "//iframe[@id='entryIframe']")
    driver.switch_to.frame(iframe)

In [148]:
driver.get("https://map.naver.com/p/search/%EC%84%B1%EC%88%98%EB%8F%99%20%EB%86%80%EA%B1%B0%EB%A6%AC?c=12.00,0,0,0,dh")
time.sleep(5)

store_list_iframe()

In [149]:
# 스크롤 컨테이너 이름 
# id = "_pcamp_list_scroll_container"
# class name = "Ryr1F"
# div#_pcamp_list_scroll_container.Ryr1F = 390 * 500
scroll_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
 
 # 해당 요소의 전체 스크롤 가능한 높이를 반환
last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
print(last_height)

stores = []

while True:
    # 요소 내에서 아래로 600px 스크롤 
    driver.execute_script("arguments[0].scrollTop += 600;", scroll_element)
    
    time.sleep(1)
    
    # 새 높이 계산
    new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
    
    # 스크롤이 더 늘어나지 않으면 종료 
    if new_height == last_height:
        break
    
    last_height = new_height
    
# a href 페이지 번호 버튼 - 첫 페이지 가게만 가져오기
# class name = "mBN2s qxokY"
page_first = driver.find_element(By.XPATH, '//a[contains(@class, "mBN2s qxokY")]').text

# 첫 페이지에 등록된 모든 가게 조회 - li 안 추출하기 
store_list = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')

print(f'현재 페이지 = {page_first}, 가게 개수 = {len(store_list)}')

for index, e in enumerate(store_list): 
    store_name = e.find_element(By.CLASS_NAME, "YgcU0").find_element(By.XPATH, './/a/div/div/span[@class="xBZDS"]')
    store_category = e.find_element(By.CLASS_NAME, "YgcU0").find_element(By.XPATH, './/a/div/div/span[@class="LF32u"]')

    print(str(index) +". " + store_name.text + "/" + store_category.text)
    
    stores.append((store_name, store_category))

3417
현재 페이지 = 1, 가게 개수 = 70
0. 성수문화예술마당/복합문화공간
1. 디뮤지엄/미술관
2. 뚝섬미술관/미술관
3. 섬세이 테라리움/전시관
4. 성수아트홀/공연장
5. 서울숲 씨어터/공연장
6. 무비랜드/영화관
7. 갤러리까르찌나/갤러리,화랑
8. LCDC SEOUL/복합문화공간
9. 코사이어티/복합문화공간
10. 무신사 테라스 성수/복합문화공간
11. entry55 Seongsu/공연,연극시설
12. 언더스탠드에비뉴/복합문화공간
13. 더서울라이티움/전시관
14. 피치스 도원/복합문화공간
15. 성수연방/복합문화공간
16. 더페이지갤러리/갤러리,화랑
17. 헤르만아트홀/공연장
18. 성수동성당/천주교
19. 레트르 성수/공방
20. 메종 릴리앙/공방
21. 성수향수공방 닷노트/공방
22. 라오크 성수/공방
23. 히포캠퍼스 성수/공방
24. 감정선프로젝트 베이킹 성수점/공방
25. 얼루브 반지공방 성수/공방
26. 위에트 성수/공방
27. 반지공방 아뜰리에호수 성수/공방
28. 플라비 성수점/공방
29. 성수향수공방 뤼미에르퍼퓸 성수/공방
30. 무드잉/공방
31. 딥아틀리에/공방
32. 모어 성수/공방
33. 토로토로 세라믹 스튜디오 성수본점/공방
34. 포토오브제 성수점/셀프,대여스튜디오
35. 휘월루 성수/공방
36. 페인트래빗 성수/공방
37. 감정선프로젝트 성수점/공방
38. 하우알유두잉 성수점 셀프사진관 셀프스튜디오/셀프,대여스튜디오
39. 크래프트 프랙티스/공방
40. 레트르 서울숲/공방
41. 셀프사진관 잇츠치즈타임 X 포토포레 성수 셀프스튜디오/
42. 디셈브레 성수/공방
43. 폼폼 스튜디오/공방
44. 라쿵 베이커리/공방
45. 케이퍼 스튜디오/공방
46. 모노맨션 성수점/셀프,대여스튜디오
47. 성수 웯우드스튜디오 나무반지공방/공방
48. 오브유/공방
49. 센트마리아쥬/공방
50. 컬러온스튜디오/공방
51. 유니유 성수/공방
52. 클라쎄 성수/공방
53. 스냅파이 서울숲/셀프,대여스튜디오
54. 영카이브 성수점/셀프

In [150]:
for name, category in stores[:10]:
    print(name.text + "/" + category.text)
    # 상점 클릭
    name.click()
    # 오른쪽 iframe으로 전환 - 상점 상세정보로 이동 
    store_info_iframe() 
    
    # 리뷰 개수 가져오기 - 방문자 리뷰, 블로그 리뷰 개수 둘다 가져오기 
    review_count =  driver.find_element(By.CLASS_NAME, "dAsGb").find_elements(By.XPATH, './/span[contains(@class, "PXMot")]/a')
    [print(r.text) for r in review_count]

    # 목록 바의 리뷰 element 가져오기
    # Tab bar가 미술관은 예매, 정보 같은 것들이 추가로 생성돼 indexing 불가 
    review_element = driver.find_element(By.CLASS_NAME, "ngGKH").find_element(By.XPATH, './/div/div/a/span[text()="리뷰"]')
    print(review_element.text)
    
    # 리뷰 탭 클릭
    review_element.click()
    
    print("##리뷰 탭 클릭##")
    
    time.sleep(2) # 로딩 필요
    
    more_contents = driver.find_elements(By.XPATH, '//div[contains(@class, "lfH3O")]/a')
    if more_contents:
        driver.execute_script("arguments[0].click();", more_contents[0])
        print("##더보기 버튼 클릭##")
        time.sleep(2)
    
    review_more_button = driver.find_elements(By.XPATH, '//div[contains(@class, "place_section_content")]/ul/li/div[contains(@class, "pui__vn15t2")]/a[contains(@class, "pui__wFzIYl")]')
    
    if review_more_button:
        driver.execute_script("arguments[0].click();", review_more_button[0])
        print("##리뷰 더보기 버튼 클릭##")
        time.sleep(1)
    
    review_contents = driver.find_elements(By.XPATH, '//div[contains(@class, "place_section_content")]/ul/li/div[contains(@class, "pui__vn15t2")]/a')
    

    [print(content.text) for content in review_contents]
    
    
    # 다시 왼쪽 iframe으로 전환해야 가게 이름 클릭 가능
    store_list_iframe()
     

성수문화예술마당/복합문화공간
방문자 리뷰 25
블로그 리뷰 578
리뷰
##리뷰 탭 클릭##
##더보기 버튼 클릭##
주차하기 좋고 화장실도 깨끗해요🙂
좋아요
주차장 완전 넓고 화장실도 좋아요
서울숲구경하러왔어요 임시주차장이용했네요 푸르는숲과더불어 튤립도피었고 시원하게 구경잘하고갑니다
굿
푸에르자 부르타 웨이라 ➡️ 잔혹한 힘 (바람ver)
360도에서 펼쳐지는 환상적인 Show
무대 연출. 에너지. 어메이징👏👏👏
편리합니더
좋아요
굳
굿
거꾸로집 사진찍기 좋아요
굿
좋아용
굿
서울은 주차요금 비쌉니다 ㅠㅠㅠ
굿
굿
주차장 넓어서 좋았습니다.
잘 주차하고 숲가서 잘 놀았습니다~
최고에요 !!
디뮤지엄/미술관
방문자 리뷰 3,252
블로그 리뷰 10,298
리뷰
##리뷰 탭 클릭##
##더보기 버튼 클릭##
##리뷰 더보기 버튼 클릭##
공간,시각을 어쩜 저리 할수있는지 독특한 방식이 산뜻했다 생각된다. 깔끔하면서 가구의 조합이 너무 좋았다.청바지소재를 잘라 천과 합작한 쇼파는 따뜻해보이며 친근감이 가서
좋았고 중정느낌의 나무느낌이 포근함을 주었다.
접기
힘들지만 그만큼 볼 게 많아 재밌습니다. 티켓할인해서 걱정을 조금 했었는데 좋았습니다. 쇼룸과 가구, 그림, 조각 등 작품이 어우러져 흥미로웠고, 중간중간 쇼파에 정말 앉고 싶고 침대에 눕고 싶었지만 잘 참고, 즐겁게 구경했습니다. 편안한 복장 추천해요!
더보기
디뮤지엄 용산에서 봤다가 성수동으로 옮기고 나서는 처음 보는 전시입니다. 👀인테리어적인 부분을 참고하기에도 너무 괜찮구요. 집과 작품들이 어우러지는 분위기를 잘 느낄 수 있었습니다.
더보기
기대없이 갔는데 의외로 볼게 많았습니다. 3개 층이라 은근 시간이 걸렸어요. 일본작가 그림이 많습니다. 그림 외 가구 인테리어도 나름 좋았습니다. 너무 붐비지 않아서 좋았고 안내해주시는 직원분들도 친절했어요. 그런데 기념품 매장에 엽서 등 살게 생각보다 별로 없어서 아쉬웠어요.
더보기
취향. M2 티 소믈리에의 집.
타인의 집에 가기 어려운 시대에 누군가의 취향을 온